In [1]:
!pip install keras_tuner

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import zipfile
import os
import tensorflow as tf
import shutil

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow import keras
import keras_tuner as kt
from keras_tuner.engine import hypermodel
from tensorflow.keras.losses import MeanSquaredError, MeanAbsoluteError, MeanAbsolutePercentageError
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.metrics import MeanSquaredError, MeanAbsoluteError, MeanAbsolutePercentageError
from tensorflow.keras.layers import Input, Dropout, Dense, Flatten, Conv1D, MaxPooling1D, AveragePooling1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.initializers import GlorotUniform, HeUniform, LecunUniform, RandomNormal

'c:\\Users\\iaioa\\Desktop\\HUB\\Not_Working_WandB'

In [4]:
# number_of_models_cc, min_max_scaler_X_cc = my_tuning(X_orig_cc, y_cc, agro_id.split()[1], 'cc')

# The function i check is the above so i need the inputs


# I rename them to the names the function uses
# REMEMBER this is only for the cc ones... 
# I load the numpy arrays from the drive

X_orig = np.load('../Data_transfer/transferX_orig.npy')
y = np.load('../Data_transfer/transfery.npy')
agro_id = '129'
model_str = 'cc'

# Import the dicts that i will make the predictions
import pickle
with open('../Data_transfer/X_to_validate_dict_df.pkl', 'rb') as f:
    X_to_validate_dict_df = pickle.load(f)

with open('../Data_transfer/y_to_validate_dict_df.pkl', 'rb') as f:
    y_to_validate_dict_df = pickle.load(f)

In [5]:
# split our data into train and validation and shuffle them 
# το train test split ειναι συνάρτηση του keras
X_train_orig, X_val_orig, y_train, y_val = train_test_split(X_orig, y, 
                                                            test_size=0.15, # και εδώ μπορώ να παίξω 
                                                            shuffle=True,
                                                            random_state=42,)

In [6]:
# By normalizing you make sure a variable doesn't have more significance than another
min_max_scaler_X = MinMaxScaler()
X_train = min_max_scaler_X.fit_transform(X_train_orig)
X_val = min_max_scaler_X.transform(X_val_orig)

In [7]:
%%capture
!pip install wandb

In [8]:
import wandb
from wandb.keras import WandbCallback

In [9]:
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\iaioa/.netrc


True

In [10]:
class MyHyperModel(hypermodel.HyperModel):
  
  # don't try to figure out what's going on into this function
  def __init__(
        self,
        number_of_layers=1,
        **kwargs):
    super(MyHyperModel, self).__init__(**kwargs)
    self.number_of_layers = number_of_layers


  def build(self, hp):

    """
    Builds the ANN (Artificial Neural Network) model. Here I set what I want to
    tune. In this particular case I tune:
        * the number of layers
        * the number of units pre layer

      Returns:
        the sequential model
    """

    model = keras.Sequential()
    model.add(Flatten())


    # Tune the number of layers.
    for i in range(hp.Int("num_layers", self.number_of_layers, self.number_of_layers)):
        model.add(
            Dense(
                # Tune number of units separately.
                units=hp.Int(f"units_{i}", min_value=16, max_value=550, step=32),
                activation=hp.Choice("activation", ["relu"]),
            )
        )
    # output layer
    model.add(Dense(1, activation="relu"))

    model.compile(
        optimizer=Adam(),
        loss="mae",
        metrics=["mae"],
    )

    return model

# Here is where i get my problems
The guide i was following had 

self.save_model(trial.trial_id, model)

in the run_trial() function but it was not running with it and only when i got it comented out i could upload my tuning at WandB

But this way afterwards when i wanted to get_best_models() there was an error thrown


**NotFoundError: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for DAM_Aggr_tuning_129_cc/DAM_Aggr_tuning_129_cc_project/trial_2/checkpoint**

In [11]:
class MyTuner(kt.Tuner):

    def run_trial(self, trial, trainX, trainY, batch_size, objective, epochs, validation_data):

        hp = trial.hyperparameters
        objective_name_str = objective

        ## create the model with the current trial hyperparameters
        model = self.hypermodel.build(hp)
        
        ## Initiates new run for each trial on the dashboard of Weights & Biases
        run = wandb.init(project="Let's see Hello2", config=hp.values)
        checkpoint_filepath=f'./Aggr_{int(agro_id)}_{model_str}/checkpoint'
        model_checkpoint_callback = ModelCheckpoint(filepath=checkpoint_filepath,
                                  monitor='val_mae',    
                                  save_best_only=True)

        my_callbacks = [
                        EarlyStopping(patience=20),
                        model_checkpoint_callback,
                        WandbCallback(), 
                      ]

        ## WandbCallback() logs all the metric data such as
        ## loss, accuracy and etc on dashboard for visualization
        history = model.fit(trainX,
                  trainY,
                  batch_size=batch_size,
                  epochs=epochs,
                  validation_data=validation_data,
                  callbacks=my_callbacks)  

        ## if val_accurcy used, use the val_accuracy of last epoch model which is fully trained
        val_mae = history.history['val_mae'][-1]  ## [-1] will give the last value in the list

        ## Send the objective data to the oracle for comparison of hyperparameters
        self.oracle.update_trial(trial.trial_id, {objective_name_str:val_mae})

        ## save the trial model
        # If i activate this function i get an error after the first trial
        # self.save_model(trial.trial_id, model)
        
        ## ends the run on the Weights & Biases dashboard
        run.finish()

In [12]:
EPOCHS = 10
BATCH_SIZE = 32 
objective = 'val_mae'



D = X_train.shape[1]    # number of features
no_model = 0    # initialize the number of models to save
parent_dir = f"{int(agro_id)}_Aggr_4_models_{model_str}"

try:
  os.makedirs(parent_dir)  # create the path to save the model
except:
  pass


for number_of_layers in range(3, 8):
  # create the model from the class MyHyperModel
  my_model =  MyHyperModel(number_of_layers)

  tuner = MyTuner(
      hypermodel=my_model,
      oracle=kt.oracles.BayesianOptimization(
        objective=objective,
        max_trials=4),
      executions_per_trial=1,
      overwrite=True,
      directory=f"DAM_Aggr_tuning_{int(agro_id)}_{model_str}",
      project_name=f"DAM_Aggr_tuning_{int(agro_id)}_{model_str}_project",
  )

  tuner.search(X_train, y_train, 
                batch_size=BATCH_SIZE,
                epochs=EPOCHS, 
                validation_data=(X_val, y_val),
                objective=objective,
                )
  # continue
  best_models = tuner.get_best_models(num_models=7)
  
  for best_model in best_models:
      best_model.build(input_shape=(None, D))   # build the model χρειάζομαι το build για να φτοιάξω το μοντέλο actualy
      saved_model_path = os.path.join(parent_dir, f"{no_model}")
      best_model.save(filepath=saved_model_path)    # save the model
      no_model += 1    

Trial 4 Complete [00h 01m 21s]
val_mae: 8.42139720916748

Best val_mae So Far: 8.42139720916748
Total elapsed time: 00h 04m 41s
INFO:tensorflow:Oracle triggered exit


NotFoundError: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for DAM_Aggr_tuning_129_cc\DAM_Aggr_tuning_129_cc_project\trial_3\checkpoint

# Get predictions

In [ ]:
best_models = []
best_val_scores = []
parent_dir = f"{int(agro_id)}_Aggr_4_models_{model_str}"

# iterate through the days in the validation set
for day_index in range(len(X_to_validate_dict_df)):
  val_scores = []
  models = []
  # iterate over the saved models
  for n in range(no_model):   # Here i changed that to match my code on the colab
      # load the model and append it to the models list
      saved_model_path = os.path.join(parent_dir, f"{n}")
      loaded_model = tf.keras.models.load_model(saved_model_path)
      models.append(loaded_model)

      # evaluate each model on the validation set
      val_scores.append(loaded_model.evaluate(min_max_scaler_X.transform(X_to_validate_dict_df[day_index].to_numpy()), y_to_validate_dict_df[day_index])[1])
  
  # for each day keep the best model, which is the one that gives the 
  # minimum validation score
  min_pos = np.where(val_scores == np.array(val_scores).min())[0][0]
  best_model = models[min_pos]
  best_val_score = val_scores[min_pos]

  # Therefore for each day of the validation set we get a different model
  best_models.append(best_model)
  best_val_scores.append(best_val_score)